<a href="https://colab.research.google.com/github/omkenge/cookbook/blob/test/quickstarts/Audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Audio Quickstart

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Audio.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

This notebook provides an example of how to prompt Gemini Flash using an audio file. In this case, you'll use a [sound recording](https://www.jfklibrary.org/asset-viewer/archives/jfkwha-006) of President John F. Kennedy’s 1961 State of the Union address.

### Install dependencies

In [1]:
%pip install -q -U google-genai

Note: you may need to restart the kernel to use updated packages.


In [2]:
from google import genai

### Configure your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](../quickstarts/Authentication.ipynb) for an example.

In [3]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

client = genai.Client(api_key=GOOGLE_API_KEY)

## Upload an audio file with the File API

To use an audio file in your prompt, you must first upload it using the [File API](../quickstarts/File_API.ipynb).


In [4]:
URL = "https://storage.googleapis.com/generativeai-downloads/data/State_of_the_Union_Address_30_January_1961.mp3"

In [5]:
!wget -q $URL -O sample.mp3

In [6]:
your_file = client.files.upload(file='sample.mp3')

## Use the file in your prompt

In [7]:
response = client.models.generate_content(
  model='gemini-2.5-flash-preview-04-17',
  contents=[
    'Listen carefully to the following audio file. Provide a brief summary.',
    your_file,
  ]
)

print(response.text)

The audio is President John F. Kennedy's State of the Union address to a joint session of Congress on January 30th, 1961.

In the speech, Kennedy describes the current moment as a time of national peril and opportunity, highlighting significant domestic and international challenges. He details the troubled state of the U.S. economy, including recession, high unemployment, slow growth, and falling farm income, and proposes a program to stimulate recovery, address unemployment, and promote growth. He also discusses the balance of payments deficit and outlines steps to strengthen the dollar and promote exports without resorting to protectionism or abandoning overseas commitments.

Kennedy addresses the global challenges, including communist pressures in Asia and Latin America, the crisis in the Congo, and the state of alliances in Europe. He emphasizes the need to strengthen both military and economic tools, proposing a re-evaluation of defense strategy and a more effective foreign aid pr

## Inline Audio

For small requests you can inline the audio data into the request, like you can with images. Use PyDub to trim the first 10s of the audio:

In [8]:
%pip install -Uq pydub

Note: you may need to restart the kernel to use updated packages.


In [9]:
from pydub import AudioSegment

In [10]:
sound = AudioSegment.from_mp3("sample.mp3")

In [11]:
sound[:10000] # slices are in ms

Add it to the list of parts in the prompt:

In [12]:
from google.genai import types

response = client.models.generate_content(
  model='gemini-2.5-flash-preview-04-17',
  contents=[
    'Describe this audio clip',
    types.Part.from_bytes(
      data=sound[:10000].export().read(),
      mime_type='audio/mp3',
    )
  ]
)

print(response.text)

The audio clip contains spoken speech describing the President's State of the Union address to a joint session of the Congress from the rostrum of the House of Representatives.


Note the following about providing audio as inline data:

- The maximum request size is 20 MB, which includes text prompts, system instructions, and files provided inline. If your file's size will make the total request size exceed 20 MB, then [use the File API](https://ai.google.dev/gemini-api/docs/audio?lang=python#upload-audio) to upload files.
- If you're using an audio sample multiple times, it is more efficient to [use the File API](https://ai.google.dev/gemini-api/docs/audio?lang=python#upload-audio).


## Get a transcript of the audio file
To get a transcript, just ask for it in the prompt. For example:


In [13]:
prompt = "Generate a transcript of the speech."

## Refer to timestamps in the audio file
A prompt can specify timestamps of the form `MM:SS` to refer to particular sections in an audio file. For example:

In [18]:
# Create a prompt containing timestamps.
prompt = "Provide a transcript of the speech between the timestamps 02:30 and 03:29."

response = client.models.generate_content(
  model='gemini-2.5-flash-preview-04-17',
  contents=[
    prompt,
    your_file,
  ]
)

print(response.text)

I speak today in an hour of national peril and national opportunity. Before my term has ended, we shall have to test anew whether a nation organized and governed such as ours can endure. The outcome is by no means certain. The answers are by no means clear. All of us together, this administration, this Congress, this nation must forge those answers. But today were I to offer, after little more than a week in office, detailed legislation to remedy every national ill, the Congress would rightly wonder whether the desire for speed had replaced the duty of responsibility. My remarks therefore will be limited, but they will also be candid. To state the facts frankly, is not to despair the future, nor to indict the past.


## Count audio tokens

You can count the number of tokens in your audio file using the [count_tokens](https://googleapis.github.io/python-genai/#count-tokens-and-compute-tokens) method.

Audio files have a fixed per second token rate (more details in the dedicated [count token quickstart](./Counting_Tokens.ipynb).

In [19]:
count_tokens_response = client.models.count_tokens(
    model='gemini-2.5-flash-preview-04-17',
    contents=[your_file],
)

print("Audio file tokens:",count_tokens_response.total_tokens)

Audio file tokens: 83528


Youtube Example

In [20]:
from google.genai import types
from IPython.display import display, Markdown
youtube_url = "https://www.youtube.com/watch?v=RDOMKIw1aF4" # Repalce with the youtube url you want to analyze

prompt = """
    Analyze the following YouTube video content. Provide a concise summary covering:

    1.  **Main Thesis/Claim:** What is the central point the creator is making?
    2.  **Key Topics:** List the main subjects discussed, referencing specific examples or technologies mentioned (e.g., AI models, programming languages, projects).
    3.  **Call to Action:** Identify any explicit requests made to the viewer.
    4.  **Summary:** Provide a concise summary of the video content.

    Use the provided title, chapter timestamps/descriptions, and description text for your analysis.
"""
# Analyze the video
response = client.models.generate_content(
    model="gemini-2.5-flash-preview-04-17",
    contents=types.Content(
        parts=[
            types.Part(text=prompt),
            types.Part(
                file_data=types.FileData(file_uri=youtube_url)
            )
        ]
    )
)
display(Markdown(response.text))

Based on the provided information:

1.  **Main Thesis/Claim:** The creator claims that Google's newly released Gemini 2.5 Pro is the best coding AI they have used so far.
2.  **Key Topics:**
    *   Evaluation of Gemini 2.5 Pro's coding capabilities through various projects.
    *   Attempting to build a complex **Ultimate Tic-Tac-Toe game in Java using Swing**.
    *   Attempting to build a **Kitten Cannon style game using p5.js**.
    *   Attempting to build **UI layouts from image mockups** (a landing page using **Vite/React/Tailwind CSS** and a static **HTML** clone of the X/Twitter website).
    *   Testing Gemini 2.5 Pro's ability to **refactor existing code** (specifically Rust code using iterators).
    *   Reviewing **performance benchmarks** of Gemini 2.5 Pro compared to other AI models (OpenAI, Claude, Grok, DeepSeek) across categories like Reasoning & Knowledge, Science, Mathematics, Code Generation, Code Editing, Agentic Coding, Factuality, Visual Reasoning, Image Understanding, and Long Context.
    *   Discussing the **currency of Gemini's training data** and its use of **Grounding with Google Search** to get up-to-date information (e.g., on the current React JS version).
3.  **Call to Action:** The creator explicitly asks viewers to share their thoughts and experiences using Gemini 2.5 Pro in the comments section. He also includes standard calls to subscribe, like, and turn on notifications.
4.  **Summary:** The video evaluates Google's Gemini 2.5 Pro, with the creator asserting it's the best coding AI they've encountered. This evaluation is based on practical coding tests, including building a Java Swing Ultimate Tic-Tac-Toe game (which was successful), a p5.js game clone (requiring fixes), and attempting to generate UI from image mockups (with mixed results, struggling particularly with a React landing page). A key highlight is Gemini's proficiency in refactoring Rust code using idiomatic iterators. The creator also examines benchmark data comparing Gemini 2.5 Pro against competing models across various tasks, noting strong performance, especially in areas like long context understanding. The video also briefly touches upon the model's ability to retrieve current information using Google Search grounding. The creator concludes by reiterating his positive impression based on these tests and asks viewers to share their own experiences.

## Next Steps
### Useful API references:

More details about Gemini API's [vision capabilities](https://ai.google.dev/gemini-api/docs/vision) in the documentation.

If you want to know about the File API, check its [API reference](https://ai.google.dev/api/files) or the [File API](https://github.com/google-gemini/cookbook/blob/main/quickstarts/File_API.ipynb) quickstart.

### Related examples

Check this example using the audio files to give you more ideas on what the gemini API can do with them:
* Share [Voice memos](https://github.com/google-gemini/cookbook/blob/main/examples/Voice_memos.ipynb) with Gemini API and brainstorm ideas

### Continue your discovery of the Gemini API

Have a look at the [Audio](../quickstarts/Audio.ipynb) quickstart to learn about another type of media file, then learn more about [prompting with media files](https://ai.google.dev/tutorials/prompting_with_media) in the docs, including the supported formats and maximum length for audio files. .
